In [1]:
import os
import glob
import pandas as pd
import numpy as np
from datetime import datetime
import time

In [2]:
variable_path_base = r'../data/variables'
variable_files_pattern = f'var-dia-*.zip'
aggregate_func = 'median'
YEARS_MIN_TO_FILL = 10

In [3]:
variable_path_base = variable_path_base.split('/')
variable_files = glob.glob(os.path.join(*variable_path_base, variable_files_pattern))
print('Archivos a cargar', *variable_files, sep = '\n')

Archivos a cargar
..\data\variables\var-dia-PTPM_CON-fill-median-10.zip
..\data\variables\var-dia-PTPM_CON.zip
..\data\variables\var-dia-PT_10_TT_D-fill-median-10.zip
..\data\variables\var-dia-PT_10_TT_D.zip
..\data\variables\var-dia-Q_MEDIA_D-fill-median-10.zip
..\data\variables\var-dia-Q_MEDIA_D.zip
..\data\variables\var-dia-Q_MN_D-fill-median-10.zip
..\data\variables\var-dia-Q_MN_D.zip
..\data\variables\var-dia-Q_MX_D-fill-median-10.zip
..\data\variables\var-dia-Q_MX_D.zip
..\data\variables\var-dia-TMN_CON-fill-median-10.zip
..\data\variables\var-dia-TMN_CON.zip
..\data\variables\var-dia-TMX_CON.zip


In [4]:
def read_variable(f):
    dtypes = { 'CodigoEstacion':'category','Etiqueta':'category'}
    dateparse = lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M')
    
    df = pd.read_csv(f, dtype = dtypes, parse_dates = ['Fecha'], date_parser = dateparse)
    return df

In [5]:
%%time
f = variable_files[1]
#for f in variable_files[3:4]:
start_time = time.time()

print(f'Archivo: {f}')
print(f'Leyendo h:{time.asctime()}...')

df = read_variable(f)
etiqueta = df.iat[0, df.columns.get_loc('Etiqueta')]
print(f'Etiqueta:{etiqueta}')

df.drop('Etiqueta', axis = 'columns', inplace = True)

print(f'**** r:{df.shape} h:{time.asctime()} t:{(time.time() - start_time) / 60} ****')

Archivo: ..\data\variables\var-dia-PTPM_CON.zip
Leyendo h:Tue Jun 16 23:35:13 2020...
Etiqueta:PTPM_CON
**** r:(13764209, 3) h:Tue Jun 16 23:41:25 2020 t:6.2084759672482805 ****
Wall time: 6min 12s


In [6]:
%%time

idx_names = ['Fecha', 'CodigoEstacion']
estaciones = df.CodigoEstacion.unique()
fechas = pd.date_range(start = df.Fecha.min(), end = df.Fecha.max())

idx = pd.MultiIndex.from_product((fechas, estaciones), names = idx_names)
df = df.set_index(idx_names).reindex(idx)
print(f'**** r:{df.shape} h:{time.asctime()} t:{(time.time() - start_time) / 60} ****')

**** r:(18694560, 1) h:Tue Jun 16 23:52:49 2020 t:17.605197183291118 ****
Wall time: 11min 23s


In [7]:
%%time
df['MesDia'] = df.index.get_level_values('Fecha').strftime('%m-%d')
df.set_index('MesDia', append = True, inplace = True)
df = df.reorder_levels(order=['MesDia', 'Fecha', 'CodigoEstacion'])
df.sort_index(inplace = True)
print(f'**** r:{df.shape} h:{time.asctime()} t:{(time.time() - start_time) / 60} ****')

**** r:(18694560, 1) h:Tue Jun 16 23:56:22 2020 t:21.148309008280435 ****
Wall time: 3min 32s


In [8]:
%%time
#df.unstack('CodigoEstacion').unstack('MesDia').fillna(lambda x: np.NAN if x.count() < YEARS_MIN_TO_FILL else x.mean())
df2 = df.unstack('CodigoEstacion')
df2.sort_index(inplace = True)

Wall time: 13.9 s


In [9]:
print(df.memory_usage(deep=True).sum() / 1E6, df2.memory_usage(deep=True).sum() / 1E6)

262.031156 149.708209


In [10]:
%%time
res = []

#start_time = time.time()
print(f'h:{time.asctime()}')

gb = df2.groupby(['MesDia'])
for g, gdf in gb:
    #print(f'g:{g} h:{time.asctime()}')
    gdf = gdf.transform(lambda x: x.fillna(np.NAN if x.count() < YEARS_MIN_TO_FILL else x.agg(aggregate_func)))
    res.append(gdf)
    #print(f'**** h:{time.asctime()} t:{(time.time() - start_time)} ****')
    
df2 = pd.concat(res)
print(f'**** r:{df2.shape} h:{time.asctime()} t:{(time.time() - start_time) / 60} ****')

h:Tue Jun 16 23:56:38 2020
**** r:(10744, 1740) h:Wed Jun 17 00:05:42 2020 t:30.48522911866506 ****
Wall time: 9min 4s


In [11]:
df2 = (pd.concat([df2], keys=[etiqueta], names=['Etiqueta'])
        .stack('CodigoEstacion', dropna = False))
print(f'**** r:{df2.shape} h:{time.asctime()} t:{(time.time() - start_time) / 60} ****')

**** r:(18694560, 1) h:Wed Jun 17 00:05:43 2020 t:30.50816241502762 ****


In [12]:
%%time
df2 = (df2.reorder_levels(order = ['CodigoEstacion', 'Fecha', 'MesDia', 'Etiqueta'])
    .sort_index()
    .reset_index()
    .drop('MesDia', axis = 'columns'))
print(f'**** r:{df2.shape} h:{time.asctime()} t:{(time.time() - start_time) / 60} ****')

**** r:(18694560, 4) h:Wed Jun 17 00:06:19 2020 t:31.094377998510996 ****
Wall time: 35.1 s


In [13]:
p = os.path.join(*variable_path_base, f'var-dia-{etiqueta}-fill-{aggregate_func}-{YEARS_MIN_TO_FILL}.csv')
print(p)
df2.to_csv(p, index = False)

..\data\variables\var-dia-PTPM_CON-fill-median-10.csv
